In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False

In [ ]:
path = "https://raw.githubusercontent.com/fenago/tf/main/Chapter9-Recurrent%20Neural%20Networks/datasets/energydata_complete.csv"

In [ ]:
df = pd.read_csv(path)

In [ ]:
df.head()

In [ ]:
df.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
df.info()

In [ ]:
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)

In [ ]:
df.set_index('date')[['Appliances', 'lights', 'T_out', 'RH_1', 'Visibility']].plot(subplots=True)

In [ ]:
df_input = df[['Appliances', 'lights', 'T_out', 'RH_1', 'Visibility']]

In [ ]:
df_input.head()

In [ ]:
df_input.describe()

In [ ]:
df_input.query('Appliances > 500')

In [ ]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df_input)

In [ ]:
X = []
y = []

In [ ]:
for i in range(60, scaled_data.shape[0]):
    X.append(scaled_data [i-60:i])
    y.append(scaled_data [i, 0])

In [ ]:
import numpy as np

X, y = np.array(X), np.array(y)

In [ ]:
X_train = X[:19000]
y_train = y[:19000]
X_test = X[19000:]
y_test = y[19000:]

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [ ]:
regressor = Sequential()
regressor.add(LSTM(units= 20, activation = 'relu',\
                   return_sequences = True,\
                   input_shape = (X_train.shape[1], X_train.shape[2])))
regressor.add(Dropout(0.5))
regressor.add(LSTM(units= 40, \
                   activation = 'relu', \
                   return_sequences = True))
regressor.add(Dropout(0.5))
regressor.add(LSTM(units= 80, \
                   activation = 'relu'))
regressor.add(Dropout(0.5))
regressor.add(Dense(units = 1))  

In [ ]:
regressor.summary()

In [ ]:
regressor.compile(optimizer='adam', loss = 'mean_squared_error')

In [ ]:
regressor.fit(X_train, y_train, epochs=2, batch_size=32)

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(y_test[-60:], color = 'black', \
         label = "Real Appliance Energy Consumption")
plt.plot(y_pred[-60:], color = 'gray', \
         label = 'Predicted Appliance Energy Consumption')
plt.title('Appliance Energy Consumption Prediction')
plt.xlabel('time')
plt.ylabel('Appliance Energy Consumption')
plt.legend()
plt.show()